<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/SQL/DS_SQL_03_NB_Windows_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Bibliografía

- [Youtube base](https://www.youtube.com/watch?v=UNuocI3KYG0&list=PLrJvjnSL5aF5oHPN82BPzn-gz_9Tpljve&index=1&t=1912s)
- [Github base](https://github.com/sqlinsights/MiercolesDeSQL/blob/master/WindowFunctions/WindowFunctions.ipynb)

# B. Librerías

In [ ]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=b1322b9d59e39c7843f36510c8ea72d0269d29fbe087052b243378b803debb4d
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [ ]:
import pandas as pd
from pandasql import sqldf

# C. Dataframe base

In [ ]:
dict_input = {
  "id_vuelo": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  "fecha_salida": ['2022-01-01', '2022-01-03', '2022-01-02', '2022-01-04', '2022-01-01', '2022-01-05', '2022-01-07', '2022-01-06', '2022-01-08', '2022-01-02'],
  "fecha_llegada": ['2022-01-02', '2022-01-04', '2022-01-03', '2022-01-05', '2022-01-02', '2022-01-06', '2022-01-08', '2022-01-07', '2022-01-09', '2022-01-03'],
  "aeronave": ['Airbus A320', 'Airbus A320', 'Boeing 737', 'Airbus A320', 'Boeing 737', 'Boeing 747', 'Airbus A320', 'Boeing 737', 'Airbus A320', 'Boeing 747'],
  "aeropuerto_origen": ['JFK', 'JFK', 'LAX', 'LHR', 'CLI', 'JFK', 'JFK', 'LAX', 'LAX', 'JFK'], 
  "tarifa": [100, 120, 150, 200, 120, 200, 200, 120, 100, 400], 
}

vuelos = pd.DataFrame.from_dict(dict_input)
vuelos

,id_vuelo,fecha_salida,fecha_llegada,aeronave,aeropuerto_origen,tarifa
0,1,2022-01-01,2022-01-02,Airbus A320,JFK,100
1,2,2022-01-03,2022-01-04,Airbus A320,JFK,120
2,3,2022-01-02,2022-01-03,Boeing 737,LAX,150
3,4,2022-01-04,2022-01-05,Airbus A320,LHR,200
4,5,2022-01-01,2022-01-02,Boeing 737,CLI,120
5,6,2022-01-05,2022-01-06,Boeing 747,JFK,200
6,7,2022-01-07,2022-01-08,Airbus A320,JFK,200
7,8,2022-01-06,2022-01-07,Boeing 737,LAX,120
8,9,2022-01-08,2022-01-09,Airbus A320,LAX,100
9,10,2022-01-02,2022-01-03,Boeing 747,JFK,400


## 01. `ROW_NUMBER()`

Principales usos: 
- Permiter enumerar secuancias por partición.
- Permite generar identificadores únicos.
- Permite identificar duplicados en una tabla.

In [ ]:
sqldf("""
SELECT id_vuelo, aeronave, aeropuerto_origen, tarifa,
       ROW_NUMBER() OVER (ORDER BY  aeropuerto_origen, aeronave) AS secuencia_row_number
FROM vuelos; 
""")

,id_vuelo,aeronave,aeropuerto_origen,tarifa,secuencia_row_number
0,5,Boeing 737,CLI,120,1
1,1,Airbus A320,JFK,100,2
2,2,Airbus A320,JFK,120,3
3,7,Airbus A320,JFK,200,4
4,6,Boeing 747,JFK,200,5
5,9,Airbus A320,LAX,100,6
6,3,Boeing 737,LAX,150,7
7,8,Boeing 737,LAX,120,8
8,4,Airbus A320,LHR,200,9
9,10,Boeing 747,LHR,400,10


In [ ]:
sqldf("""
SELECT id_vuelo, aeronave, aeropuerto_origen, tarifa,
       ROW_NUMBER() OVER (PARTITION BY aeropuerto_origen, aeronave) AS secuencia_row_number
FROM vuelos; 
""")

,id_vuelo,aeronave,aeropuerto_origen,tarifa,secuencia_row_number
0,5,Boeing 737,CLI,120,1
1,1,Airbus A320,JFK,100,1
2,2,Airbus A320,JFK,120,2
3,7,Airbus A320,JFK,200,3
4,6,Boeing 747,JFK,200,1
5,9,Airbus A320,LAX,100,1
6,3,Boeing 737,LAX,150,1
7,8,Boeing 737,LAX,120,2
8,4,Airbus A320,LHR,200,1
9,10,Boeing 747,LHR,400,1


In [ ]:
sqldf("""
SELECT id_vuelo, aeronave, aeropuerto_origen, tarifa,
       ROW_NUMBER() OVER (PARTITION BY aeropuerto_origen ORDER BY aeronave) AS secuencia_row_number
FROM vuelos; 
""")

,id_vuelo,aeronave,aeropuerto_origen,tarifa,secuencia_row_number
0,5,Boeing 737,CLI,120,1
1,1,Airbus A320,JFK,100,1
2,2,Airbus A320,JFK,120,2
3,7,Airbus A320,JFK,200,3
4,6,Boeing 747,JFK,200,4
5,9,Airbus A320,LAX,100,1
6,3,Boeing 737,LAX,150,2
7,8,Boeing 737,LAX,120,3
8,4,Airbus A320,LHR,200,1
9,10,Boeing 747,LHR,400,2


## 02. `NTILE()`

Principales usos: 
- Permite dividir un grupo de datos en conjuntos con igual número de registros.

In [ ]:
sqldf("""
SELECT id_vuelo, aeronave, aeropuerto_origen, tarifa,
       NTILE(3) OVER (ORDER BY tarifa) AS secuencia_row_number
FROM vuelos; 
""")

,id_vuelo,aeronave,aeropuerto_origen,tarifa,secuencia_row_number
0,1,Airbus A320,JFK,100,1
1,9,Airbus A320,LAX,100,1
2,2,Airbus A320,JFK,120,1
3,5,Boeing 737,CLI,120,1
4,8,Boeing 737,LAX,120,2
5,3,Boeing 737,LAX,150,2
6,4,Airbus A320,LHR,200,2
7,6,Boeing 747,JFK,200,3
8,7,Airbus A320,JFK,200,3
9,10,Boeing 747,LHR,400,3


## 03. `LEAD() & LAG()`

Principales usos: 
- Permite obtener valores previos y posteriores en una secuancia.

In [ ]:
sqldf("""
SELECT id_vuelo, fecha_salida, aeropuerto_origen, tarifa,
       LAG(tarifa) OVER (ORDER BY tarifa) AS lag_ecuencia_row_number, 
       LEAD(tarifa) OVER (ORDER BY tarifa) AS lead_secuencia_row_number
FROM vuelos; 
""")

,id_vuelo,fecha_salida,aeropuerto_origen,tarifa,lag_ecuencia_row_number,lead_secuencia_row_number
0,1,2022-01-01,JFK,100,NaN,100.0
1,9,2022-01-08,LAX,100,100.0,120.0
2,2,2022-01-03,JFK,120,100.0,120.0
3,5,2022-01-01,CLI,120,120.0,120.0
4,8,2022-01-06,LAX,120,120.0,150.0
5,3,2022-01-02,LAX,150,120.0,200.0
6,4,2022-01-04,LHR,200,150.0,200.0
7,6,2022-01-05,JFK,200,200.0,200.0
8,7,2022-01-07,JFK,200,200.0,400.0
9,10,2022-01-02,LHR,400,200.0,NaN


## 04. `LAST_VALUE() & FIRST_VALUE()`

Principales usos: 
- Permite obtener el primer y último valor de una particion determinada.

In [ ]:
sqldf("""
SELECT id_vuelo, fecha_salida, aeropuerto_origen, tarifa,
       FIRST_VALUE(tarifa) OVER (PARTITION BY aeropuerto_origen ORDER BY id_vuelo) AS first_secuencia_row_number, 
       LAST_VALUE(tarifa) OVER (PARTITION BY aeropuerto_origen ORDER BY id_vuelo
    RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS LastSale
FROM vuelos; 
""")

,id_vuelo,fecha_salida,aeropuerto_origen,tarifa,first_secuencia_row_number,LastSale
0,5,2022-01-01,CLI,120,120,120
1,1,2022-01-01,JFK,100,100,200
2,2,2022-01-03,JFK,120,100,200
3,6,2022-01-05,JFK,200,100,200
4,7,2022-01-07,JFK,200,100,200
5,3,2022-01-02,LAX,150,150,100
6,8,2022-01-06,LAX,120,150,100
7,9,2022-01-08,LAX,100,150,100
8,4,2022-01-04,LHR,200,200,400
9,10,2022-01-02,LHR,400,200,400


## 05. `RANK()` & `DENSE_RANK()` & `PERCENT_RANK()`

In [ ]:
sqldf("""
SELECT id_vuelo, fecha_salida, aeropuerto_origen, tarifa,
       RANK() OVER (ORDER BY tarifa DESC) AS rank,
       DENSE_RANK() OVER (ORDER BY tarifa DESC) AS dense_rank,
       PERCENT_RANK() OVER (ORDER BY tarifa) AS dense_rank
FROM vuelos; 
""")

,id_vuelo,fecha_salida,aeropuerto_origen,tarifa,rank,dense_rank,dense_rank
0,10,2022-01-02,JFK,400,1,1,1.000000
1,4,2022-01-04,LHR,200,2,2,0.666667
2,6,2022-01-05,JFK,200,2,2,0.666667
3,7,2022-01-07,JFK,200,2,2,0.666667
4,3,2022-01-02,LAX,150,5,3,0.555556
5,2,2022-01-03,JFK,120,6,4,0.222222
6,5,2022-01-01,CLI,120,6,4,0.222222
7,8,2022-01-06,LAX,120,6,4,0.222222
8,1,2022-01-01,JFK,100,9,5,0.000000
9,9,2022-01-08,LAX,100,9,5,0.000000


## 06. `CUME_DISC()` (Distribución acomulada)



Principales usos: 
- Ver la distribución ordinal.

In [ ]:
sqldf("""
SELECT id_vuelo, fecha_salida, aeropuerto_origen, tarifa,
       CUME_DIST() OVER (ORDER BY tarifa) AS dense_rank
FROM vuelos; 
""")

,id_vuelo,fecha_salida,aeropuerto_origen,tarifa,dense_rank
0,1,2022-01-01,JFK,100,0.2
1,9,2022-01-08,LAX,100,0.2
2,2,2022-01-03,JFK,120,0.5
3,5,2022-01-01,CLI,120,0.5
4,8,2022-01-06,LAX,120,0.5
5,3,2022-01-02,LAX,150,0.6
6,4,2022-01-04,LHR,200,0.9
7,6,2022-01-05,JFK,200,0.9
8,7,2022-01-07,JFK,200,0.9
9,10,2022-01-02,JFK,400,1.0


## 07. `SUM()` (Funciones de agregación)

In [ ]:
sqldf("""
SELECT aeropuerto_origen, aeronave, id_vuelo, tarifa,
       SUM(tarifa) OVER (PARTITION BY aeropuerto_origen ORDER BY aeronave) AS query
FROM vuelos; 
""")

,aeropuerto_origen,aeronave,id_vuelo,tarifa,query
0,CLI,Boeing 737,5,120,120
1,JFK,Airbus A320,1,100,420
2,JFK,Airbus A320,2,120,420
3,JFK,Airbus A320,7,200,420
4,JFK,Boeing 747,6,200,1020
5,JFK,Boeing 747,10,400,1020
6,LAX,Airbus A320,9,100,100
7,LAX,Boeing 737,3,150,370
8,LAX,Boeing 737,8,120,370
9,LHR,Airbus A320,4,200,200


In [ ]:
sqldf("""
SELECT aeropuerto_origen, aeronave, id_vuelo, tarifa,
       SUM(tarifa) OVER (PARTITION BY aeropuerto_origen) AS query
FROM vuelos; 
""")

,aeropuerto_origen,aeronave,id_vuelo,tarifa,query
0,CLI,Boeing 737,5,120,120
1,JFK,Airbus A320,1,100,1020
2,JFK,Airbus A320,2,120,1020
3,JFK,Boeing 747,6,200,1020
4,JFK,Airbus A320,7,200,1020
5,JFK,Boeing 747,10,400,1020
6,LAX,Boeing 737,3,150,370
7,LAX,Boeing 737,8,120,370
8,LAX,Airbus A320,9,100,370
9,LHR,Airbus A320,4,200,200
